In [1]:
from dfdc.core.core import *
from dfdc.core.video_core import *

In [2]:
from fastai.vision import *
from tqdm import tqdm

In [3]:
data_path = Path("/home/ubuntu/data/dfdc/")

In [4]:
detections_path = data_path/'dfdc_face_detections'

In [5]:
len(detections_path.ls()), detections_path.ls()[0]

(50,
 PosixPath('/home/ubuntu/data/dfdc/dfdc_face_detections/part_38_retina_detections.csv'))

In [6]:
croppped_path = data_path/'dfdc_cropped_faces'

In [7]:
len(croppped_path.ls()), croppped_path.ls()[0]

(50, PosixPath('/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc_train_part_37'))

### 1) Read detections df and crop files

In [8]:
from dfdc.core.core import *

In [9]:
part_no = '4'

In [10]:
detections_df = read_face_detection_df(detections_path/f'part_{part_no}_retina_detections.csv')

In [11]:
detections_df.head()

,fname,label,split,original,source,size,face_detections,n_frames,sample_freq,len_video
0,oyixebfpcl.mp4,FAKE,train,bgpoldvzrh.mp4,bgpoldvzrh.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[1300, 351, 15...",30,10,299
1,shclgsfxtj.mp4,FAKE,train,bgpoldvzrh.mp4,bgpoldvzrh.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[1300, 351, 15...",30,10,299
2,mutuhmwjdv.mp4,FAKE,train,bgpoldvzrh.mp4,bgpoldvzrh.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[1300, 351, 15...",30,10,299
3,rohakxryar.mp4,FAKE,train,bgpoldvzrh.mp4,bgpoldvzrh.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[1300, 351, 15...",30,10,299
4,fpiybwcszz.mp4,FAKE,train,bgpoldvzrh.mp4,bgpoldvzrh.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[1300, 351, 15...",30,10,299


In [12]:
detections_df['face_detections'][0]

[{'frame_no': 0,
  'detections': [[1300, 351, 1555, 588]],
  'landmarks': [[1333, 441, 1439, 444, 1349, 477, 1327, 525, 1407, 527]]},
 {'frame_no': 10,
  'detections': [[1306, 351, 1555, 580]],
  'landmarks': [[1338, 439, 1437, 441, 1349, 473, 1333, 521, 1408, 522]]},
 {'frame_no': 20,
  'detections': [[1312, 349, 1551, 566]],
  'landmarks': [[1336, 436, 1429, 438, 1343, 472, 1333, 515, 1404, 516]]},
 {'frame_no': 30,
  'detections': [[1312, 337, 1561, 558]],
  'landmarks': [[1338, 422, 1436, 425, 1345, 456, 1334, 504, 1410, 505]]},
 {'frame_no': 40,
  'detections': [[1326, 317, 1567, 542], [373, 360, 698, 696]],
  'landmarks': [[1354, 397, 1452, 405, 1355, 431, 1337, 479, 1414, 485],
   [557, 490, 651, 490, 663, 549, 577, 614, 650, 615]]},
 {'frame_no': 50,
  'detections': [[393, 358, 685, 674], [1327, 301, 1575, 526]],
  'landmarks': [[580, 483, 625, 476, 688, 530, 603, 596, 643, 592],
   [1367, 383, 1470, 384, 1382, 408, 1351, 462, 1435, 463]]},
 {'frame_no': 60,
  'detections': [[1

In [231]:
detections_df.shape

(1701, 10)

In [242]:
for dets in detections_df['face_detections']: 
    if [o['detections'] for o in dets] == []: break

In [252]:
cropped_part_path = croppped_path/f'dfdc_train_part_{part_no}'

In [267]:
empty_dirs = [o for o in cropped_part_path.ls() if (o.ls()) == []] # no detections - false positives

In [250]:
cropped_files = get_files(cropped_part_path, extensions=['.jpg'], recurse=True)

In [251]:
len(cropped_files)

40087

In [246]:
cropped_files[:4]

[PosixPath('/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc_train_part_4/bmgcyjggxe/frame_22_30.jpg'),
 PosixPath('/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc_train_part_4/bmgcyjggxe/frame_14_30.jpg'),
 PosixPath('/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc_train_part_4/bmgcyjggxe/frame_10_30.jpg'),
 PosixPath('/home/ubuntu/data/dfdc/dfdc_cropped_faces/dfdc_train_part_4/bmgcyjggxe/frame_20_30.jpg')]

### 2) Collect and sort frames for each video

In [247]:
videoframes_dict = defaultdict(list)

In [248]:
for o in cropped_files: videoframes_dict[o.parent.name].append("/".join(str(o).split("/")[-3:]))

In [249]:
len(videoframes_dict)

1544

In [114]:
for k,v in videoframes_dict.items(): 
    new_v = sorted(v, key=lambda o: int(Path(o).name.split("_")[1]))
    videoframes_dict[k] = new_v

In [115]:
res = []
for k,v in videoframes_dict.items(): res.append({"fname":f"{k}.mp4", "face_crop_fnames":v})
face_crop_fnames_df = pd.DataFrame(res)

In [116]:
face_crop_fnames_df.head()

,fname,face_crop_fnames
0,fopjiyxiqd.mp4,"[dfdc_train_part_0/fopjiyxiqd/frame_1_30.jpg, ..."
1,zqtvyuoabj.mp4,"[dfdc_train_part_0/zqtvyuoabj/frame_1_30.jpg, ..."
2,dwkkqhxclz.mp4,"[dfdc_train_part_0/dwkkqhxclz/frame_1_30.jpg, ..."
3,rmmcypdfbr.mp4,"[dfdc_train_part_0/rmmcypdfbr/frame_1_30.jpg, ..."
4,hkvcbcfulf.mp4,"[dfdc_train_part_0/hkvcbcfulf/frame_1_30.jpg, ..."


In [117]:
source2fakes = defaultdict(list)
for _, row in detections_df.iterrows():
    if row['source'] != row['fname']: source2fakes[row['source']].append(row['fname'])

In [118]:
print(f"Total # of original videos: {len(source2fakes)}")

Total # of original videos: 86


### 3) Compare original and fake frame pixels

It is discussed in forums that there is at least 1 pixel change in every video

In [184]:
def plot_itemlist(il):
    n = int(np.ceil(np.sqrt(len(il))))
    axes = subplots(n,n).flatten()
    for i, ax in zip(range(len(il)), axes): il[i].show(ax=ax)

In [192]:
# identical_idxs_dict = {}
# for source, fakes in source2fakes.items(): 
#     # get original frames itemlist
#     video_name = Path(source).stem
#     frame_fnames = videoframes_dict[video_name]
#     original_il = ImageList([croppped_path/o for o in frame_fnames])

#     # detect if there are unchanged fake frame  
#     identical_idxs = []
#     for i in range(len(fakes)):
#         video_name = Path(fakes[i]).stem
#         frame_fnames = videoframes_dict[video_name]
#         fake_il = ImageList([croppped_path/o for o in frame_fnames])
#         identical_idxs.append((np.where([torch.equal(orig_frame.data, fake_frame.data) 
#                            for orig_frame, fake_frame in zip(original_il, fake_il)])))
    
#     # 
#     identical_idxs_dict[source] = identical_idxs

In [ ]:
# plot_itemlist(original_il)
# plot_itemlist(fake_il)

In [16]:
part_no = '0'

In [277]:
os.makedirs(data_path/'dfdc_training_csv', exist_ok=True)

In [278]:
def create_training_df(part_no):
    # get cropped files
    detections_df = pd.read_csv(detections_path/f'part_{part_no}_retina_detections.csv')
    cropped_files = get_files(croppped_path/f'dfdc_train_part_{part_no}', extensions=['.jpg'], recurse=True)

    # collect video frames relative to parent cropped faces dir
    videoframes_dict = defaultdict(list)
    for o in cropped_files: videoframes_dict[o.parent.name].append("/".join(str(o).split("/")[-3:]))

    # sort by frame #
    for k,v in videoframes_dict.items(): 
        new_v = sorted(v, key=lambda o: int(Path(o).name.split("_")[1]))
        videoframes_dict[k] = new_v

    # create dataframe
    res = []
    for k,v in videoframes_dict.items(): res.append({"fname":f"{k}.mp4", "face_crop_fnames":v})
    face_crop_fnames_df = pd.DataFrame(res)

    train_df = detections_df.merge(face_crop_fnames_df, on='fname', how='inner')
    train_df.to_csv(data_path/f'dfdc_training_csv/part_{part_no}_training.csv', index=False)
    return train_df

In [279]:
parts = list(map(str, range(50)))

In [280]:
for part in parts: create_training_df(part)

In [281]:
train_df

,fname,label,split,original,source,size,face_detections,n_frames,sample_freq,len_video,face_crop_fnames
0,zumqqvixhu.mp4,FAKE,train,hntguogkqd.mp4,hntguogkqd.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[894, 135, 105...",30,10,299,"[dfdc_train_part_1/zumqqvixhu/frame_1_30.jpg, ..."
1,zrrncwxmdc.mp4,FAKE,train,hntguogkqd.mp4,hntguogkqd.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[894, 135, 105...",30,10,299,"[dfdc_train_part_1/zrrncwxmdc/frame_1_30.jpg, ..."
2,hntguogkqd.mp4,REAL,train,NaN,hntguogkqd.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[894, 135, 105...",30,10,299,"[dfdc_train_part_1/hntguogkqd/frame_1_30.jpg, ..."
3,slhkaosehs.mp4,FAKE,train,hntguogkqd.mp4,hntguogkqd.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[894, 135, 105...",30,10,299,"[dfdc_train_part_1/slhkaosehs/frame_1_30.jpg, ..."
4,qeefkatpus.mp4,FAKE,train,hntguogkqd.mp4,hntguogkqd.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[894, 135, 105...",30,10,299,"[dfdc_train_part_1/qeefkatpus/frame_1_30.jpg, ..."
...,...,...,...,...,...,...,...,...,...,...,...
1694,nfsztvjqpk.mp4,FAKE,train,nymodlmxni.mp4,nymodlmxni.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[845, 107, 963...",30,10,300,"[dfdc_train_part_1/nfsztvjqpk/frame_1_30.jpg, ..."
1695,wlehzhjppc.mp4,FAKE,train,zfobicuigx.mp4,zfobicuigx.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[851, 83, 955,...",30,10,300,"[dfdc_train_part_1/wlehzhjppc/frame_1_30.jpg, ..."
1696,zfobicuigx.mp4,REAL,train,NaN,zfobicuigx.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[851, 83, 955,...",30,10,300,"[dfdc_train_part_1/zfobicuigx/frame_1_30.jpg, ..."
1697,qjdtgggqym.mp4,REAL,train,NaN,qjdtgggqym.mp4,"(1080, 1920)","[{'frame_no': 0, 'detections': [[880, 105, 100...",30,10,300,"[dfdc_train_part_1/qjdtgggqym/frame_1_30.jpg, ..."
